This is a scheduelr to incorporate new matches into the an excel file. 
The excel file would be reviewed and then added to db

In [2]:
import pandas as pd
from datetime import datetime
import config
import soccerdata as sd
import sql_connector
import schedule_helper as SH

C:\Users\Hp\AppData\Local\Temp\ipykernel_25952\2121265208.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


[05/13/24 02:37:21] INFO     Custom team name replacements loaded from                                _config.py:82
                             C:\Users\Hp\soccerdata\config\teamname_replacements.json.                             

                    INFO     No custom league dict found. You can configure additional leagues in    _config.py:204
                             C:\Users\Hp\soccerdata\config\league_dict.json.                                       

Import Successful


In [3]:
FOTMOB_URL = config.getFotMobUrls()
FOTMOB_LEAGUE = config.getFotMobLeagueDict()
CONFIG = config.getGeneralConfig()

In [4]:
FOTMOB_LEAGUE

{'ENG-Premier League': 47,
 'ESP-La Liga': 87,
 'ITA-Serie A': 55,
 'GER-Bundesliga': 54,
 'FRA-Ligue 1': 53,
 'INT-World Cup': 77,
 'UCL': 42,
 'UEL': 73,
 'UECL': 10216,
 'Eredivisie': 57,
 'Pro_League': 40,
 'Primeira_Liga': 61}

In [5]:
leagues = sorted(list(FOTMOB_LEAGUE))

In [6]:
def getCurrentSeasonName(dt):
    month = dt.month
    yr = dt.year
    if month <=7 :
        return f"{(yr%2000)-1}-{yr%2000}",(yr%2000)-1
    else:
        return f"{yr%2000}-{(yr%2000)+1}",(yr%2000)

In [7]:
league = leagues[0]
season,df_season = getCurrentSeasonName(datetime.now())
season

'23-24'

In [8]:
last_date_info = []
for x in leagues :
    print("Working for league ",x)
    query = f"""
    SELECT competition,MAX(date) as last_match_date from match_schedule
    where competition = '{x}' AND season = '{season}'
    """
    # if x == 'Eredivisie':
    #     print(query)
    #     print()
    response = sql_connector.execute_select_query(query)[0]['last_match_date']
    if response:
        last_date_info.append({ x : response})


Working for league  ENG-Premier League
Working for league  ESP-La Liga
Working for league  Eredivisie
Working for league  FRA-Ligue 1
Working for league  GER-Bundesliga
Working for league  INT-World Cup
Working for league  ITA-Serie A
Working for league  Primeira_Liga
Working for league  Pro_League
Working for league  UCL
Working for league  UECL
Working for league  UEL


In [9]:
last_date_info

[{'ENG-Premier League': datetime.date(2024, 5, 6)},
 {'ESP-La Liga': datetime.date(2024, 5, 10)},
 {'Eredivisie': datetime.date(2024, 5, 6)},
 {'FRA-Ligue 1': datetime.date(2024, 5, 10)},
 {'GER-Bundesliga': datetime.date(2024, 5, 10)},
 {'ITA-Serie A': datetime.date(2024, 5, 10)},
 {'Primeira_Liga': datetime.date(2024, 5, 10)},
 {'Pro_League': datetime.date(2024, 5, 5)},
 {'UCL': datetime.date(2024, 5, 8)},
 {'UEL': datetime.date(2024, 5, 9)}]

In [10]:
# item = last_date_info[-3:-2]
# item

In [11]:
def fetch_and_merge_current_season_dfs(league,current_season):
    WS_df,FB_df,FM_df = SH.fetch_all_dfs_for_season_of_league(league, latest_season=current_season)
    WS,FB,FM = SH.make_dfs_uniform(league,WS_df,FB_df,FM_df)
    team_alias = SH.get_team_alias()
    return SH.merge_and_save_df(league,WS,FB,FM,team_alias)
# df = fetch_and_merge_current_season_dfs(p[0],df_season)

In [12]:
def fetch_games_since_last(df,last_date,overall):
    DF = df
    DF['date'] = pd.to_datetime(DF['date']).dt.date
    DF = DF[DF['date'] >= last_date]
    col_str = "("+",".join(list(DF.columns))+")"
    queries = []
    temp_d = DF
    temp_d['date'] = temp_d['date'].apply(lambda x: x.strftime('%d-%b-%Y'))
    temp_d = temp_d[['competition','home_team','away_team','score','date']]
    ov_df = pd.conact([overall,temp_d])
    for i,(ind,x) in enumerate(DF.iterrows()):
        row = DF.iloc[i]
        whoscored_url = row['whoscored_url']
        fbref_url = row['fbref_url']
        fotmob_url = row['fotmob_url']
        
        all_urls_present = bool(whoscored_url) and bool(fbref_url) and bool(fotmob_url)
        
        if all_urls_present :
            value_str = "('"+"','".join(map(str, x))+"')"
            q = f'INSERT INTO match_schedule {col_str} VALUES {value_str}'.replace('.0','')
            queries.append((whoscored_url,fbref_url,fotmob_url,q))
        else:
            print("All IDs are not present : ",row['home_team'],row['away_team'],row['date'])
            print("WS: ",bool(whoscored_url),"FB: ",bool(fbref_url),"FM: ",bool(fotmob_url))
            print()
    return queries,ov_df

In [14]:
insert_queries = []
err_leagues = []
d = None
for item in last_date_info[3:4]:
    # print(item)
    # continue
    try :
        league_time_info = tuple(item.items())[0]
        df = fetch_and_merge_current_season_dfs(league_time_info[0],df_season)
        overall_df = pd.DataFrame() # EMpty df to add all the rows here
        new_insert_queries,overall_df = fetch_games_since_last(df,league_time_info[1],overall_df)
        insert_queries+=new_insert_queries
    except Exception as e :
        err_leagues.append(item)

[05/13/24 02:40:11] INFO     Saving cached data to C:\Users\Hp\soccerdata\data\WhoScored              _common.py:91

[05/13/24 02:40:17] INFO     patching driver executable                                              patcher.py:346
                             C:\Users\Hp\appdata\roaming\undetected_chromedriver\undetected_chromedr               
                             iver.exe                                                                              

[05/13/24 02:40:20] INFO     Saving cached data to C:\Users\Hp\soccerdata\data\FBref                  _common.py:91

[05/13/24 02:40:37] INFO     Scraping game schedule from                                           whoscored.py:428
                             https://www.whoscored.com/Regions/74/Tournaments/22/Seasons/9635/Fran                 
                             ce-Ligue-1                                                                            

[05/13/24 02:40:38] INFO     Scraping game schedule for Ijumaa, Mei 3 2024                         whoscored.py:327

...

                    INFO     Scraping game schedule for Jumapili, Ago 20 2023                      whoscored.py:327

[05/13/24 02:42:20] INFO     Scraping game schedule for Ijumaa, Ago 25 2023                        whoscored.py:327

[05/13/24 02:42:21] INFO     Scraping game schedule for Jumamosi, Ago 26 2023                      whoscored.py:327

                    INFO     Scraping game schedule for Jumapili, Ago 27 2023                      whoscored.py:327

Returning 1 DFs combination
Preparing Fetched DataFrames ... 
FRA-Ligue 1
Merging Fetched DataFrames ... 
(306, 9)


In [29]:
d = df

In [30]:
d['date'] = d['date'].apply(lambda x: x.strftime('%d-%b-%Y'))
d = d[['competition','home_team','away_team','score','date']]

In [31]:
d

,competition,home_team,away_team,score,date
0,FRA-Ligue 1,Nice,Lille,1–1,11-Aug-2023
1,FRA-Ligue 1,Marseille,Reims,2–1,12-Aug-2023
2,FRA-Ligue 1,Paris Saint-Germain,Lorient,0–0,12-Aug-2023
3,FRA-Ligue 1,Brest,Lens,3–2,13-Aug-2023
4,FRA-Ligue 1,Clermont Foot,Monaco,2–4,13-Aug-2023
...,...,...,...,...,...
283,FRA-Ligue 1,Metz,Rennes,2–3,04-May-2024
284,FRA-Ligue 1,Brest,Nantes,0–0,04-May-2024
285,FRA-Ligue 1,Lille,Lyon,3–4,06-May-2024
286,FRA-Ligue 1,Brest,Reims,1–1,10-May-2024


In [ ]:
D = d[['competition','home_team','away_team','score','date']]

In [ ]:
# import smtplib
# import ssl
# from email.mime.text import MIMEText
# from email.message import EmailMessage

#     # Set up the MIME
# def send_email(sender_email, receiver_email, subject, message, password):
#     # Create a message object
#     msg = EmailMessage()
#     msg["From"] = sender_email
#     msg["To"] = receiver_email
#     msg["Subject"] = subject

#     # Add body to email
#     msg.set_content(message)

#     # Create a secure SSL context
#     context = ssl.create_default_context()

#     with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
#         server.login(sender_email, password)
#         server.send_message(msg)
#         print("Done")

# # Example usage
# sender_email = "mranalust@gmail.com"
# receiver_email = "aakashkaushal75@example.com"
# subject = "Scheduler Script Info"
# message = "This is a test email sent from Python."
# password = 'lzms qlxo grbg matr'
# # password = '(Superpass,1234)'

# send_email(sender_email, receiver_email, subject, message, password)


In [14]:
err_ind = []
dfs = []
qqq = []
for i,(WS_url,FB_url,FM_url,q) in enumerate(insert_queries):
    try:
        fetch_query = f"select * from match_schedule where whoscored_url like'%{WS_url}%' or"
        fetch_query += f" fbref_url like'%{FB_url}%' or fotmob_url like '%{FM_url}%'; "
        fetch_query = fetch_query.replace(".0",'')
        l = sql_connector.execute_select_query(fetch_query)
        print("Done Query No : ",i+1,len(l))
        
        if len(l) !=0:
            if len(l) > 1 :
                dfs.append(pd.DataFrame(l))
            continue
        else:
            qqq.append(q)
        sql_connector.execute_insert_query(q)
        # print(l,type(l))
        print()
    except Exception as e:
        err_ind.append((i,str(e)))

Done Query No :  1 1
Done Query No :  2 1
Done Query No :  3 1
Done Query No :  4 1
Done Query No :  5 1
Done Query No :  6 1
Done Query No :  7 1
Done Query No :  8 1
Done Query No :  9 1
Done Query No :  10 1
Done Query No :  11 0

Done Query No :  12 0

Done Query No :  13 0

Done Query No :  14 0

Done Query No :  15 0

Done Query No :  16 0

Done Query No :  17 0

Done Query No :  18 0

Done Query No :  19 0

Done Query No :  20 0

Done Query No :  21 0

Done Query No :  22 0

Done Query No :  23 0

Done Query No :  24 0

Done Query No :  25 0

Done Query No :  26 0

Done Query No :  27 0

Done Query No :  28 0

Done Query No :  29 0

Done Query No :  30 0

Done Query No :  31 0

Done Query No :  32 0

Done Query No :  33 0

Done Query No :  34 0

Done Query No :  35 0

Done Query No :  36 0

Done Query No :  37 0

Done Query No :  38 0

Done Query No :  39 0

Done Query No :  40 0

Done Query No :  41 0

Done Query No :  42 0

Done Query No :  43 0

Done Query No :  44 0

Done Qu

In [17]:
qqq

["INSERT INTO match_schedule (competition,season,stage,date,home_team,away_team,score,whoscored_id,whoscored_url,fbref_id,fbref_url,fotmob_id,fotmob_url) VALUES ('ITA-Serie A','23-24','None','2024-03-30','Napoli','Atalanta','0–3','1746348','https://1xbet.whoscored.com/Matches/1746348/Live/Italy-Serie-A-2023-2024-Napoli-Atalanta','45e6d215','/en/matches/45e6d215/Napoli-Atalanta-March-30-2024-Serie-A','4230839','/matches/atalanta-vs-ssc-napoli/2ss9sz#4230839')",
 "INSERT INTO match_schedule (competition,season,stage,date,home_team,away_team,score,whoscored_id,whoscored_url,fbref_id,fbref_url,fotmob_id,fotmob_url) VALUES ('ITA-Serie A','23-24','None','2024-03-30','Genoa','Frosinone','1–1','1746352','https://1xbet.whoscored.com/Matches/1746352/Live/Italy-Serie-A-2023-2024-Genoa-Frosinone','949865c6','/en/matches/949865c6/Genoa-Frosinone-March-30-2024-Serie-A','4230841','/matches/frosinone-vs-genoa/3ckfw9#4230841')",
 "INSERT INTO match_schedule (competition,season,stage,date,home_team,away

In [ ]:
if err_ind :
    html = """
    Error Occured
    """
else:
    send sucess mail